### Importing the required Libraries

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
import os 
from supabase import create_client, Client

### Initialising a pre-trained YOLO model to detect people

In [2]:
person_model = YOLO("yolov8m.pt")

# persons = person_model("imagess.jpg", classes=[0])

### Loading our previously trained Bed Detection model

In [3]:
bed_model = YOLO("D:/runs/detect/train3/weights/best.pt")

##### Calculate the Intersection over Union (IoU) between two bounding boxes of two models

In [4]:
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA) #intersection area

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea #union area

    return interArea/float(unionArea) if unionArea > 0 else 0

##### Detect beds and people

#### determine if a person is on a bed or not by utlising our IOU function on bounding boxes

In [5]:
def process_frame(frame, frame_id=0):
    bed_results = bed_model(frame)
    person_results = person_model(frame, classes=[0])  
    
    bed_boxes = [list(map(int, box.xyxy[0].tolist())) for box in bed_results[0].boxes]
    person_boxes = [list(map(int, box.xyxy[0].tolist())) for box in person_results[0].boxes]
    # Using only person detection model on video makes the video render even faster since it is the pre-trained model

    frame_arr = np.array(frame)

    occupied_beds = set()
    assigned_beds = set()
    
    print(f"[Frame {frame_id}] Total Beds Detected: {len(bed_boxes)}")
    print(f"[Frame {frame_id}] Total Persons Detected: {len(person_boxes)}")

    for person in person_boxes:
        person_x1, person_y1, person_x2, person_y2 = person

        # Ignoring people standing
        person_width = person_x2 - person_x1
        person_height = person_y2 - person_y1
        if person_height > 1.8 * person_width:
            print(f"Person {person} ignored (dude was standing)")
            continue

        best_bed_idx = None
        max_iou = 0

        for bed_idx, bed in enumerate(bed_boxes):
            if bed_idx in assigned_beds:  
                continue  # Skip if this bed is already assigned to another person

            overlap_score = iou(person, bed)  # Comparing the bounding boxes
            print(f"IOU Person {person} vs Bed {bed} = {overlap_score:.2f}")

            if overlap_score >= 0.08 and overlap_score > max_iou:  
                max_iou = overlap_score
                best_bed_idx = bed_idx

        if best_bed_idx is not None:
            occupied_beds.add(best_bed_idx)
            assigned_beds.add(best_bed_idx)
            print(f"Person {person} assigned to the Bed {bed_boxes[best_bed_idx]}")
            cv2.rectangle(frame, (bed_boxes[best_bed_idx][0], bed_boxes[best_bed_idx][1]), (bed_boxes[best_bed_idx][2], bed_boxes[best_bed_idx][3]), (0, 0, 255), 2) 
            # Draw red rectangle on occupied beds (don't know why this not working yet)

    # Draw all beds and people
    for box in bed_boxes:
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)  # Blue for beds
    for box in person_boxes:
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)  # Green for persons
        
    return len(occupied_beds), len(bed_boxes), frame

##### Handling both imagery and motion pictures with the same function by passing frames as parameters to the above function

In [6]:
def detect_bed_occupancy(media_path):
    if media_path.lower().endswith(('.jpg', '.jpeg', '.png')):  # Process image formats pnly (initial stages)
        
        img = cv2.imread(media_path)
        if img is None:
            print(f"Error: Could not load image: {media_path}")
            return 0

        occupied_count, total_beds, output_img = process_frame(img)
        print(f"Occupied Beds: {occupied_count}/{total_beds}")

        cv2.imshow("Bed Occupancy Detection", output_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        return abs(total_beds - occupied_count)

    elif media_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):  # Process a video (absolute necessity for handling CCTV footages)
        
        cap = cv2.VideoCapture(media_path)
        
        if not cap.isOpened() :
            print(f"Error: Could not open video: {media_path}")
            return 0

        frame_id = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            occupied_count, total_beds, output_frame = process_frame(frame, frame_id)
            cv2.imshow("Bed Occupancy Detection", output_frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            frame_id += 1

        cap.release()
        cv2.destroyAllWindows()

        return abs(total_beds - occupied_count)

    else:
        print("Unsupported file type")
        return -1
    
    

#     print(f"Occupied Beds: {len(occupied_beds)}")

#     return abs(len(occupied_beds) - len(bed_boxes))

### Testing the final product

In [12]:
filename = "vid_footage.mp4" ## change the file type as per requirement

free_beds = detect_bed_occupancy(filename)

print(free_beds)


0: 640x640 1 hospital bed, 1018.9ms
Speed: 15.6ms preprocess, 1018.9ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 989.3ms
Speed: 16.2ms preprocess, 989.3ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)
[Frame 0] Total Beds Detected: 1
[Frame 0] Total Persons Detected: 2
IOU Person [199, 1106, 1094, 1309] vs Bed [806, 856, 1440, 1440] = 0.12
Person [199, 1106, 1094, 1309] assigned to the Bed [806, 856, 1440, 1440]

0: 640x640 1 hospital bed, 1000.4ms
Speed: 16.4ms preprocess, 1000.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 persons, 974.7ms
Speed: 21.8ms preprocess, 974.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
[Frame 1] Total Beds Detected: 1
[Frame 1] Total Persons Detected: 4
IOU Person [194, 1108, 1092, 1307] vs Bed [805, 853, 1440, 1440] = 0.12
Person [194, 1108, 1092, 1307] assigned to the Bed [805, 853, 1440, 1440]
Person [500, 651, 556, 768] ignored 

[Frame 15] Total Beds Detected: 1
[Frame 15] Total Persons Detected: 6
IOU Person [157, 1113, 1075, 1306] vs Bed [790, 858, 1439, 1434] = 0.11
Person [157, 1113, 1075, 1306] assigned to the Bed [790, 858, 1439, 1434]
Person [481, 647, 527, 759] ignored (dude was standing)

0: 640x640 1 hospital bed, 963.8ms
Speed: 17.4ms preprocess, 963.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 persons, 988.9ms
Speed: 21.0ms preprocess, 988.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
[Frame 16] Total Beds Detected: 1
[Frame 16] Total Persons Detected: 6
IOU Person [153, 1112, 1069, 1307] vs Bed [789, 857, 1439, 1433] = 0.11
Person [153, 1112, 1069, 1307] assigned to the Bed [789, 857, 1439, 1433]
Person [479, 646, 525, 758] ignored (dude was standing)

0: 640x640 1 hospital bed, 973.9ms
Speed: 16.0ms preprocess, 973.9ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 persons, 946.4ms
Speed: 12.8ms preproces

### Run this cell only if you want to visualise working of two model on a video

##### We are just running two models on the video without any fps drop, no other logic is implemented here

##### Count of free beds is done only by the above python functions

In [9]:
FILENAME = 'vid_footage.mp4'     # Replace with 0 if you want to open webcam

capp = cv2.VideoCapture(FILENAME) 

if not capp.isOpened():
    print("Error opening video stream or file")
    
while True:
    ret, frame = capp.read()
    if not ret:
        print("Finished processing video.")
        break
    
    bed_results = bed_model.predict(source=frame, stream=True, verbose=False)
    person_results = person_model.predict(source=frame, stream=True, classes=[0], verbose=False)

    for result in bed_results:  # bounding boxes for beds
        boxes = result.boxes.xyxy.cpu().numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2) 
            cv2.putText(frame, "Bed", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    for result in person_results:  # bounding boxes for people
        boxes = result.boxes.xyxy.cpu().numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 
            cv2.putText(frame, "Person", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow('YOLOv8 Real-Time Bed and person detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # press "q" while the video is screening to exit the loop
        break

capp.release()
cv2.destroyAllWindows()

### Updation of the available beds in Supabase public table for realtime updates 

In [11]:
# supabase = create_client(os.getenv("supabaseUrl"), os.getenv("supabaseKey"))

supabaseUrl="URL"
supabaseKey="anon key T.T "   # use .env files to store URL and supabase anon key
supabase = create_client(supabaseUrl, supabaseKey)

if free_beds > 0:
    try:
        response = supabase.table("bed").select("*").eq("ward_id", "general").eq("empty", False).limit(free_beds).execute()

        beds_to_update = response.data

        if not beds_to_update:
            print("No beds found which empty attribute as False")

        bed_ids = [bed["bed_id"] for bed in beds_to_update]

        update_response = supabase.table("bed").update({"empty": True}).in_("bed_id", bed_ids).execute()

        print(f"Updated {free_beds} bed(s) as empty.")

    except Exception as e:
        print("Error:", e)

Updated 1 bed(s) as empty.
